# Data Preparation

In [7]:
# Load nyc_geo JSON
import json
import pandas as pd
from pandas.io.json import json_normalize
import os
import plotly.express as px

file = open('../data/nyc_geo.json')
data = json.load(file)

### Task
Parse the json **nyc_geo.json** into the dataframe with the following columns:
- Borough
- Neighborhood
- Latitude
- Longitude

In [8]:
data.keys()

dict_keys(['type', 'totalFeatures', 'features', 'crs', 'bbox'])

In [9]:
df = pd.json_normalize(data['features'])
df.head()

,type,id,geometry_name,geometry.type,geometry.coordinates,properties.name,properties.stacked,properties.annoline1,properties.annoline2,properties.annoline3,properties.annoangle,properties.borough,properties.bbox
0,Feature,nyu_2451_34572.1,geom,Point,"[-73.84720052054902, 40.89470517661]",Wakefield,1,Wakefield,None,None,0.0,Bronx,"[-73.84720052054902, 40.89470517661, -73.84720..."
1,Feature,nyu_2451_34572.2,geom,Point,"[-73.82993910812398, 40.87429419303012]",Co-op City,2,Co-op,City,None,0.0,Bronx,"[-73.82993910812398, 40.87429419303012, -73.82..."
2,Feature,nyu_2451_34572.3,geom,Point,"[-73.82780644716412, 40.887555677350775]",Eastchester,1,Eastchester,None,None,0.0,Bronx,"[-73.82780644716412, 40.887555677350775, -73.8..."
3,Feature,nyu_2451_34572.4,geom,Point,"[-73.90564259591682, 40.89543742690383]",Fieldston,1,Fieldston,None,None,0.0,Bronx,"[-73.90564259591682, 40.89543742690383, -73.90..."
4,Feature,nyu_2451_34572.5,geom,Point,"[-73.9125854610857, 40.890834493891305]",Riverdale,1,Riverdale,None,None,0.0,Bronx,"[-73.9125854610857, 40.890834493891305, -73.91..."


In [10]:
# Separate coordinates into longitude and latitude columns
longitude = []
latitude = []

for row in df['geometry.coordinates']:
    longitude.append(row[0])
    latitude.append(row[1])

df['longitude'] = longitude
df['latitude'] = latitude

In [11]:
# Tidy Data
nyc_df = df[['id','properties.borough', 'properties.name', 'longitude', 'latitude']]
nyc_df.columns = ['id', 'borough', 'neighborhood', 'longitude', 'latitude']
nyc_df

,id,borough,neighborhood,longitude,latitude
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834
...,...,...,...,...,...
301,nyu_2451_34572.302,Manhattan,Hudson Yards,-74.000111,40.756658
302,nyu_2451_34572.303,Queens,Hammels,-73.805530,40.587338
303,nyu_2451_34572.304,Queens,Bayswater,-73.765968,40.611322
304,nyu_2451_34572.305,Queens,Queensbridge,-73.945631,40.756091


In [12]:
 # csv = nyc_df.to_csv('nyc_geo_data_cleaned.csv')

### Task
Use different data sources and APIs to collect information about the neigborhoods that can be used for segmentation.

In [15]:
populations = pd.read_csv('../data/nyc_populations.csv')
populations = populations[['neighborhood', 'population']]
populations.head()

,neighborhood,population
0,Wakefield,21242.0
1,Co-op City,43752.0
2,Eastchester,11506.0
3,Fieldston,9287.0
4,Riverdale,9287.0


In [16]:
nyc_df = pd.merge(nyc_df, populations, on='neighborhood')
nyc_df = nyc_df.drop_duplicates()

In [17]:
nyc_df = nyc_df.drop(index=[120,121, 147, 148, 185, 186])

In [18]:
nyc_df.shape

(306, 6)

In [19]:
nyc_df.head()

,id,borough,neighborhood,longitude,latitude,population
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0


In [22]:
resturants = pd.read_csv('../data/API_data/resturants_data.csv')
resturants = resturants.iloc[:, 1:]
resturants.head()

,id,number_of_restaurants,average_review_count,average_restaurant_rating,average_price_level
0,nyu_2451_34572.1,13,169,4.0,1
1,nyu_2451_34572.2,12,771,4.0,2
2,nyu_2451_34572.3,19,532,4.1,1
3,nyu_2451_34572.4,9,297,4.4,1
4,nyu_2451_34572.5,14,319,4.0,2


In [23]:
nyc_df = pd.merge(nyc_df, resturants, on='id')
nyc_df = nyc_df.drop_duplicates()

In [24]:
nyc_df.head()

,id,borough,neighborhood,longitude,latitude,population,number_of_restaurants,average_review_count,average_restaurant_rating,average_price_level
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0,13,169,4.0,1
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0,12,771,4.0,2
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0,19,532,4.1,1
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0,9,297,4.4,1
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0,14,319,4.0,2


In [27]:
yelp_categories = pd.read_csv('../data/API_data/yelp_restaurant_categories.csv')
yelp_categories = yelp_categories[['id', 'yelp_category']]
yelp_categories.head()

,id,yelp_category
0,nyu_2451_34572.1,caribbean
1,nyu_2451_34572.2,delis
2,nyu_2451_34572.3,caribbean
3,nyu_2451_34572.4,N
4,nyu_2451_34572.5,burgers


In [28]:
nyc_df = pd.merge(nyc_df, yelp_categories, on='id')

In [29]:
nyc_df.head()

,id,borough,neighborhood,longitude,latitude,population,number_of_restaurants,average_review_count,average_restaurant_rating,average_price_level,yelp_category
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0,13,169,4.0,1,caribbean
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0,12,771,4.0,2,delis
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0,19,532,4.1,1,caribbean
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0,9,297,4.4,1,N
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0,14,319,4.0,2,burgers


In [30]:
# Check for missing data
total = nyc_df.isnull().sum().sort_values(ascending=False)
percent = (nyc_df.isnull().sum()/nyc_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
id,0,0.0
borough,0,0.0
neighborhood,0,0.0
longitude,0,0.0
latitude,0,0.0


In [31]:
# save_file = nyc_df.to_csv('old_dataset.csv')

In [32]:
main_dataset = nyc_df.copy()
main_dataset = main_dataset.drop(columns=['average_review_count', 'average_restaurant_rating', 'average_price_level'])
main_dataset = main_dataset.rename(columns={'number_of_restaurants': 'restaurant_count'})
main_dataset.head()

,id,borough,neighborhood,longitude,latitude,population,restaurant_count,yelp_category
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0,13,caribbean
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0,12,delis
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0,19,caribbean
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0,9,N
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0,14,burgers


In [33]:
onehot = pd.get_dummies(main_dataset['yelp_category'], prefix='', prefix_sep='')
main_dataset = main_dataset.join(onehot)
main_dataset.head()

,id,borough,neighborhood,longitude,latitude,population,restaurant_count,yelp_category,N,bars,...,pizza,russian,sandwiches,seafood,spanish,steak,sushi,tapasmallplates,thai,tradamerican
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0,13,caribbean,0,0,...,0,0,0,0,0,0,0,0,0,0
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0,12,delis,0,0,...,0,0,0,0,0,0,0,0,0,0
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0,19,caribbean,0,0,...,0,0,0,0,0,0,0,0,0,0
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0,9,N,1,0,...,0,0,0,0,0,0,0,0,0,0
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0,14,burgers,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
venues = pd.read_csv('../data/API_data/all_venues_data.csv')
venues.head()

,id,accounting,art_gallery,bakery,bank,bar,beauty_salon,bicycle_store,book_store,bowling_alley,...,store,subway_station,supermarket,synagogue,taxi_stand,tourist_attraction,transit_station,travel_agency,university,veterinary_care
0,nyu_2451_34572.1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,nyu_2451_34572.10,1,0,0,0,0,2,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,nyu_2451_34572.100,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,nyu_2451_34572.101,0,1,2,1,6,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,nyu_2451_34572.102,0,0,0,2,4,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [37]:
main_dataset = pd.merge(main_dataset, venues, on='id')
main_dataset = main_dataset.drop(columns=['id','yelp_category'], axis = 1)
main_dataset.head()

,borough,neighborhood,longitude,latitude,population,restaurant_count,N,bars,bbq,breakfast_brunch,...,store,subway_station,supermarket,synagogue,taxi_stand,tourist_attraction,transit_station,travel_agency,university,veterinary_care
0,Bronx,Wakefield,-73.847201,40.894705,21242.0,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bronx,Co-op City,-73.829939,40.874294,43752.0,12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bronx,Eastchester,-73.827806,40.887556,11506.0,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bronx,Fieldston,-73.905643,40.895437,9287.0,9,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Bronx,Riverdale,-73.912585,40.890834,9287.0,14,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


In [38]:
# save_file = main_dataset.to_csv('main_dataset.csv', index=False)

In [39]:
yelp_not_encoded_dataset = nyc_df.copy()
yelp_not_encoded_dataset= yelp_not_encoded_dataset.drop(columns=['average_review_count', 'average_restaurant_rating', 'average_price_level'])
yelp_not_encoded_dataset = pd.merge(yelp_not_encoded_dataset, venues, on='id')
yelp_not_encoded_dataset = yelp_not_encoded_dataset.drop(columns=['id'], axis = 1)
yelp_not_encoded_dataset= yelp_not_encoded_dataset.rename(columns={'number_of_restaurants': 'restaurant_count', 'yelp_category': 'most_popular_restaurant_category'})
yelp_not_encoded_dataset.head()

,borough,neighborhood,longitude,latitude,population,restaurant_count,most_popular_restaurant_category,accounting,art_gallery,bakery,...,store,subway_station,supermarket,synagogue,taxi_stand,tourist_attraction,transit_station,travel_agency,university,veterinary_care
0,Bronx,Wakefield,-73.847201,40.894705,21242.0,13,caribbean,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bronx,Co-op City,-73.829939,40.874294,43752.0,12,delis,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bronx,Eastchester,-73.827806,40.887556,11506.0,19,caribbean,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bronx,Fieldston,-73.905643,40.895437,9287.0,9,N,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Bronx,Riverdale,-73.912585,40.890834,9287.0,14,burgers,0,0,0,...,0,0,0,2,0,0,0,0,0,0


In [40]:
# save_file = yelp_not_encoded_dataset.to_csv('yelp_not_encoded.csv', index=False)

### Task

Visualize the neigborhoods in the graph.

In [43]:
def map_scatter(data, latitude='latitude', longitude='longitude', hover_name='neighborhood', color='cluster'):
    # Define mapbox token
    maptoken = os.environ["MAPBOX_TOKEN"]

    # plot points and color by cluster
    fig = px.scatter_mapbox(data, lat=latitude, lon=longitude,
                            height=500)
    fig.update_geos(fitbounds="locations")
    fig.update_layout(mapbox_style="dark", mapbox_accesstoken=maptoken, margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [44]:
map_scatter(main_dataset, latitude='latitude', longitude='longitude')